In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">
    <h2>Project Scope</h2>
        <ul>
            <li>Using solar generation and associated temperature data to explore time series forecasting.</li>
            <li>Compare the performance of ML models and a deep learning LSTM model to make a day-ahead forecast of power generation</li>
            <li>See what impact including the temperature data has on the forecast</li>
            <li>Experiment with feature engineering and test the impact differernt features have on the models</li>
            <li>Fine tune the models</li>
            <li>Report on what model is ultimately more successful and comment on possible project extensions</li>
    </ul>
</span>

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">
    <h2>Loading and exploring data</h2>
This data was taken from two solar plants in India over the course of 34 days in 15 minute intervals. The weather data is on the plant level (just measured from one sensor) and the generation data is gathered from individual inverters across the plant. More information can be found here: 
<a href="https://www.kaggle.com/datasets/anikannal/solar-power-generation-data">Data Card</a>. Credit to 
<a href="https://www.kaggle.com/anikannal">Ani Kannal</a> for uploading this dataset to Kaggle.
</span>

In [ ]:
df_plt1_gen = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv')
df_plt2_gen = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv')
df_plt1_weather = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
df_plt2_weather = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv')

df_plt1_gen.sample(10, random_state=1)

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">PLANT_ID is the same throughout, so it can be safely removed. The generation is 0 for nighttime, which of course makes sense for solar power. Also, let's rename to sources to make them easier to track</span>

In [ ]:
df_plt1_gen = df_plt1_gen.drop('PLANT_ID', axis=1)
df_plt1_gen['SOURCE_KEY'] = df_plt1_gen.SOURCE_KEY.map({df_plt1_gen.SOURCE_KEY.unique()[i-1]: f'Source_{i}' for i in range(1, len(df_plt1_gen.SOURCE_KEY.unique()) +1)})
df_plt1_gen.info()

In [ ]:
df_plt1_gen.SOURCE_KEY

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">The dtypes make sense, but we'll need to make the DATE_TIME a datetime object for easier analysis. No nulls detected here, but there still could be outliers or nulls encoded in a different way.</span>

In [ ]:
df_plt1_gen['DATE_TIME'] = pd.to_datetime(df_plt1_gen.DATE_TIME, format='%d-%m-%Y %H:%M')
df_plt1_gen.dtypes

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Now we can make some additional features out of the datetime object to aid in our analysis. If we were modeling throughout the year(s), month and year could be interesting to account for seasonal variation and long-term trends, but since our data only covers 34 days, we will omit them. We can use dayofyear to capture any longer trends that might be present.</span>

In [ ]:
df_plt1_gen['HOUR'] = df_plt1_gen.DATE_TIME.dt.hour
df_plt1_gen['DAY'] = df_plt1_gen.DATE_TIME.dt.dayofyear
df_plt1_gen['DAY_WEEK'] = df_plt1_gen.DATE_TIME.dt.dayofweek
df_plt1_gen['DAY_FULL'] = df_plt1_gen.DATE_TIME.dt.day
df_plt1_gen['MINUTES'] = df_plt1_gen.DATE_TIME.dt.time
#This maps the 15 minute intervals over the course of the day to ints 1-96. 
df_plt1_gen['MINUTES'] = df_plt1_gen.MINUTES.map({df_plt1_gen.MINUTES.unique()[i-1]:i for i in range(1, 97)})

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Next, let's check on the linear correlation of features</span>

In [ ]:
corr = df_plt1_gen.corr(numeric_only=True)
corr

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">And why not make it a heatmap</span>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Spectral')

sns.heatmap(corr, annot=True)

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Not surprisingly DC and AC power are highly correlated. This is good! It probably means the inverters in the plant are working correctly to convert the DC to AC. We will ultimately make AC_POWER our target and are probably safe to remove DC_POWER at this point. There is a strong correlation between HOUR and DAILY_YIELD, which makes sense as the daily yield increases throughout the day. Now we will look at a pairplot to see another representation of these relationships and look for any non-linear correlations.</span>

In [ ]:
df_plt1_gen = df_plt1_gen.drop('DC_POWER', axis=1)

In [ ]:
# g = sns.PairGrid(df_plt1_gen.sample(5000), diag_sharey=False, hue='SOURCE_KEY')
# g.map_upper(sns.scatterplot, s=15)
# g.map_lower(sns.kdeplot)
# g.map_diag(sns.histplot)
# #This takes quite a while to run, so saving this figure for future use.
# plt.savefig("gen_pairgrid.png", dpi=400)
from IPython.display import Image, display
display(Image(filename='/kaggle/working/gen_pairgrid.png'))

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">There is A LOT of information to take in here! The zeros dominate the distribution for AC_POWER and DAILY_YIELD due to the nighttime. It looks like some data is missing based on the DAY/MINUTES pair. The inverters start to seperate out based on TOTAL_YIELD and DAILY_YIELD. This suggests the they are operating a different capacities; probably due to any number of factors: age, location within the plant, need for maintaince, etc. Exploring this more is outside the scope of this project, but could be a good starting point for another project. Finally, the connection between HOUR and AC_POWER is interesting and also makes sense with the peaks in the middle of the day. Let's explore this more next and also start to look for outliers.</span>

## Outliers

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">To explore outliers let's start by defining an outlier as \< 1th percentile or \> 99th percentile. To do this I will make four quantile features below: 2 of them in the outlier range and 2 to establish 1 standard deviation from the mean. Then we can plot the results. Let's also get the mean while we're at it.</span>

In [ ]:
df_plt1_gen = df_plt1_gen.merge(df_plt1_gen.groupby('MINUTES').quantile(0.01, numeric_only=True).AC_POWER.rename('OUTLIERS_LOW_AC_POWER'), on='MINUTES', how='left')
df_plt1_gen = df_plt1_gen.merge(df_plt1_gen.groupby('MINUTES').quantile(0.99, numeric_only=True).AC_POWER.rename('OUTLIERS_HIGH_AC_POWER'), on='MINUTES', how='left')
df_plt1_gen = df_plt1_gen.merge(df_plt1_gen.groupby('MINUTES').mean(numeric_only=True).AC_POWER.rename('MEAN'), on='MINUTES', how='left')
df_plt1_gen = df_plt1_gen.merge(df_plt1_gen.groupby('MINUTES').std(numeric_only=True).AC_POWER.rename('STD'), on='MINUTES', how='left')
df_plt1_gen['STD_1'] = df_plt1_gen.query('AC_POWER < (MEAN + STD) and AC_POWER > (MEAN - STD)').AC_POWER

In [ ]:
fig, ax = plt.subplots()
sns.color_palette("Paired")
sns.scatterplot(data=df_plt1_gen, y='AC_POWER', x='MINUTES', hue='SOURCE_KEY', palette='gray', alpha=0.05, legend=False)
sns.scatterplot(data=df_plt1_gen, y='STD_1', x='MINUTES', hue='SOURCE_KEY', palette='gray', alpha =0.5, legend=False)
sns.scatterplot(data=df_plt1_gen.query('AC_POWER > OUTLIERS_HIGH_AC_POWER'), y='AC_POWER', x='MINUTES', hue='SOURCE_KEY', size='AC_POWER')
sns.scatterplot(data=df_plt1_gen.query('AC_POWER < OUTLIERS_LOW_AC_POWER'), y='AC_POWER', x='MINUTES', hue='SOURCE_KEY', size= -df_plt1_gen.AC_POWER) 

plt.title('AC Power per 15 minutes')
plt.ylabel('AC Power (kW)')
plt.xlabel('Hour of Day')
ax.set_xticks([i for i in range(1, 97, 4)])
ax.set_xticklabels([i for i in range(24)])
ax.legend(['Full data', '+/- 1\u03C3', 'Outliers'])

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Interestingly, it looks like there are several instances where the power generated during the middle of the day was 0. This could either indicate bad data, malfunctioning invererters, some kind of planned maintence, really cloudy days, really sunny days where the plant is overheating/at capacity or any number of other things. Without having a deeper domain knowledge it it hard to know for sure, but let's see if we can find any pattern to these mid-day outliers.</span>


<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">First, we'll check to see if any inverters in particular are responsible.

In [ ]:
outliers_low = df_plt1_gen.query('AC_POWER < OUTLIERS_LOW_AC_POWER')
outliers_high = df_plt1_gen.query('AC_POWER > OUTLIERS_HIGH_AC_POWER')

outliers_source_low = outliers_low.groupby('SOURCE_KEY').count().DATE_TIME.reset_index().sort_values('DATE_TIME', ascending=False)
outliers_source_low_zero = outliers_low[outliers_low.AC_POWER == 0].groupby('SOURCE_KEY').count().DATE_TIME.reset_index().sort_values('DATE_TIME', ascending=False)
fig, ax = plt.subplots()
sns.barplot(data = outliers_source_low, y='SOURCE_KEY', x='DATE_TIME', alpha = 0.5, )
sns.barplot(data = outliers_source_low_zero, y='SOURCE_KEY', x='DATE_TIME')
plt.title('Outlier Counts by Source (shading for 0.0 AC Power)')
plt.ylabel('Source')
plt.xlabel('Count')

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">The spread of outlier counts, including the 0.0 AC_POWER measurements are concentrated in sources 11 and 1. It could be that these were offline for an extended period due to maintance or malfunction. Below we'll look at the spread of the outliers with 0.0 AC_POWER measurements over the course of the full 34 days to see if there is any regularity.</span>

In [ ]:
outliers_zero_count = outliers_low[outliers_low.AC_POWER == 0].groupby(['SOURCE_KEY','DAY']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
outliers_zero_count
fig, ax = plt.subplots(figsize=(8,5))
sns.scatterplot(data=outliers_zero_count, x='DAY', y='COUNTS', hue='SOURCE_KEY', size='COUNTS', legend='brief')
ax.legend(bbox_to_anchor=(1, 1.05))
ax.set_xticks([i for i in range(135,170)])
ax.set_xticklabels([])


<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Several occur on the same day. There could have maintance that day or another issue.</span>

In [ ]:
outliers_zero_count = outliers_low[outliers_low.AC_POWER == 0].groupby(['SOURCE_KEY','DAY_WEEK']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
outliers_zero_count
fig, ax = plt.subplots(figsize=(8,2.5))
sns.scatterplot(data=outliers_zero_count, x='DAY_WEEK', y='COUNTS', hue='SOURCE_KEY', size='COUNTS', legend=False)
# ax.legend(label='Sources', loc='upper left')


<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Interestingly, several of the zeros occur on Sunday. This could still be coincidence or it might suggest some scheduled maintance, since Sunday is likely a day with less demand on the grid. Let's circle back and check on non-zero outliers as well.</span>

In [ ]:
outliers_low_count = outliers_low.groupby(['SOURCE_KEY','DAY_WEEK']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
fig, ax = plt.subplots(figsize=(8,2.5))
sns.scatterplot(data=outliers_low_count, x='DAY_WEEK', y='COUNTS', hue='SOURCE_KEY', size='COUNTS', legend=False)

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">There is a greater number of overall outliers on Monday. We might guess that after Sunday maintance there is a delay getting everything back online that goes into Monday. I think there is enough evidence now to choose to keep these outliers in out data as it might help the model account for some of this. That being said the number is low enough that it likely won't have too much of an impact.</span>

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">The outliers representing the 99th percentile are not as much of a concern, as they seem well connected to the distribution and likely just indicate extra sunny/hot days. We can check this assumption by seeing how they lineup with the weather data. First, we'll load and take a quick look at the weather data.</span>

## Comparing 99th percentile outliers to weather data

In [ ]:
df_plt1_weather.sample(10, random_state=1)

In [ ]:
df_plt1_weather.info()

In [ ]:
df_plt1_weather.describe(include='all').T

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">The weather data looks pretty clean. We can remove the PLANT_ID and SOURCE_KEY, since they are the same throughout (as a reminder the weather data is only measured from a single source). Again, we should make the DATE_TIME a datetime object.</span>

In [ ]:
df_plt1_weather = df_plt1_weather.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_plt1_weather['DATE_TIME'] = pd.to_datetime(df_plt1_weather.DATE_TIME, format='%Y-%m-%d %H:%M:%S') 

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Now we can join the weather data with our plant data on datetime.</span>

In [ ]:
df_plt1_gen = df_plt1_gen.merge(df_plt1_weather, on='DATE_TIME', how='left')

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">To compare the temperatures to the outliers, let's first make a outlier feature that says whether or not a row is an outlier.</span>

In [ ]:
df_plt1_gen['OUTLIER_HIGH_BOOL'] = (df_plt1_gen.AC_POWER > df_plt1_gen.OUTLIERS_HIGH_AC_POWER).astype('int')
df_plt1_gen['OUTLIER_LOW_BOOL'] = (df_plt1_gen.AC_POWER < df_plt1_gen.OUTLIERS_LOW_AC_POWER).astype('int')

In [ ]:
df_plt1_gen['YEAR'] = df_plt1_gen.DATE_TIME.dt.year
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
sns.boxplot(data=df_plt1_gen, y='AMBIENT_TEMPERATURE', x='YEAR', hue='OUTLIER_HIGH_BOOL', ax=ax1)
sns.boxplot(data=df_plt1_gen, y='AMBIENT_TEMPERATURE', x='YEAR', hue='OUTLIER_LOW_BOOL', ax=ax2, palette='rocket')
plt.title('Ambient Temperature Distribution of Outliers vs rest of data')
ax1.legend(bbox_to_anchor=(2.5, .75))
ax2.legend(bbox_to_anchor=(1, 1))

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">These results do fit with what we would expect. The outliers with high power generation are on the high end of the ambiemt temperature distribution. And like we discovered earlier, the low power generation outliers, must have an explination other than the weather, since the are even above the mean of the ambient temp distribution. To quantify this some, we can perform a quick two-sample t-test.</span>

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">For the 99th percentile outliers, we'll have a null hypothesis that the sample mean of the ambient temperature distribution across non-outliers is the same as sample mean of the ambient temperatures including the outliers; with the alternative hypothesis that the sample mean of the ambient temperature distribution across non-outliers is less than the sample mean of the ambient temperatures including the outliers. Let's have a typical threshold of 0.05.</span>

In [ ]:
from scipy import stats

stats, pval_high = stats.ttest_ind(np.array(df_plt1_gen.query('OUTLIER_HIGH_BOOL == 0').AMBIENT_TEMPERATURE), np.array(df_plt1_gen.query('OUTLIER_HIGH_BOOL == 1').AMBIENT_TEMPERATURE), alternative='less', equal_var=True, nan_policy='omit')
print("p-value for 99th percentile outliers: ", pval_high)

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">With a p-value well below our significance level, we can safely reject the null hypothesis in favor of the alternative hypothesis that the sample mean of the ambient temperature distribution across non-outliers is less than the sample mean of the ambient temperatures including the outliers.</span>

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">For the 1th percentile outliers, we'll have a null hypothesis that the sample mean of the ambient temperature distribution across non-outliers is the same as sample mean of the ambient temperatures including the outliers; with the alternative hypothesis that the sample mean of the ambient temperature distribution across non-outliers is greater than the sample mean of the ambient temperatures including the outliers. Let's again have a threshold of 0.05.</span>

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">

In [ ]:
from scipy import stats

stats, pval_low = stats.ttest_ind(np.array(df_plt1_gen.query('OUTLIER_LOW_BOOL == 0').AMBIENT_TEMPERATURE), np.array(df_plt1_gen.query('OUTLIER_LOW_BOOL == 1').AMBIENT_TEMPERATURE), alternative='greater', equal_var=True, nan_policy='omit')
print("p-value for 1th percentile outliers: ", np.round(pval_low, 4))

<span style="font-family:Roboto Mono;font-stretch:normal;font-style:normal;font-weight:200;word-spacing:-.225em;">Now the p-value is clearly above our significance level, so we accept the null hypothesis that the sample mean of the ambient temperature distribution across non-outliers is the same as the sample mean of the ambient temperatures including the outliers. This lines up with our earlier speculation.</span>